In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,65,2024-03-12,1,115.906250,97.381250,0.531828,10.003125,24.985937,0.233578,118.626563,...,11.192187,29.610937,0.197188,118.218750,0,3,1,2,23,2024
1,66,2024-03-12,1,123.092308,101.446154,0.580415,11.164615,23.427692,0.180031,121.326154,...,10.442187,20.664062,0.203422,121.303125,0,3,1,2,12,2024
2,65,2024-03-12,2,114.421875,102.470313,0.538703,11.754688,19.914062,0.173641,111.475000,...,12.823077,22.340000,0.187185,108.023077,0,3,1,2,30,2024
3,65,2024-03-12,2,113.000000,98.479688,0.524953,10.929688,24.765625,0.202672,113.917187,...,12.576563,22.820312,0.173797,111.415625,0,3,1,2,11,2024
4,64,2024-03-12,1,120.746032,97.215873,0.576397,10.544444,24.488889,0.193794,123.390476,...,12.901563,28.821875,0.211344,117.560937,0,3,1,3,2,2024
5,66,2024-03-12,1,120.923077,100.156923,0.572046,10.821538,21.489231,0.220246,120.156923,...,11.250794,23.920635,0.172159,118.142857,0,3,1,2,17,2024
6,66,2024-03-12,-2,113.123077,96.673846,0.559308,12.595385,23.064615,0.219538,116.252308,...,11.128571,24.541270,0.214016,120.687302,0,3,1,2,18,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-12,PHI,NYK,3.100000,1.387597,0
1,2024-03-12,IND,OKC,3.300000,1.350877,0
2,2024-03-12,WAS,MEM,1.769231,2.100000,1
3,2024-03-12,HOU,SAS,1.588235,2.420000,1
4,2024-03-12,BOS,UTA,1.400000,3.050000,1
5,2024-03-12,MIL,SAC,1.704225,2.200000,1
6,2024-03-12,MIN,LAC,3.250000,1.359712,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-12,PHI,NYK,3.100000,1.387597,0
1,2024-03-12,IND,OKC,3.300000,1.350877,0
2,2024-03-12,WAS,MEM,1.769231,2.100000,1
3,2024-03-12,HOU,SAS,1.588235,2.420000,1
4,2024-03-12,BOS,UTA,1.400000,3.050000,0
5,2024-03-12,MIL,SAC,1.704225,2.200000,0
6,2024-03-12,MIN,LAC,3.250000,1.359712,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-12,PHI,NYK,3.100000,1.387597,0
1,2024-03-12,IND,OKC,3.300000,1.350877,0
2,2024-03-12,WAS,MEM,1.769231,2.100000,1
3,2024-03-12,HOU,SAS,1.588235,2.420000,0
4,2024-03-12,BOS,UTA,1.400000,3.050000,0
5,2024-03-12,MIL,SAC,1.704225,2.200000,0
6,2024-03-12,MIN,LAC,3.250000,1.359712,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-12,PHI,NYK,3.100000,1.387597,0
1,2024-03-12,IND,OKC,3.300000,1.350877,0
2,2024-03-12,WAS,MEM,1.769231,2.100000,1
3,2024-03-12,HOU,SAS,1.588235,2.420000,1
4,2024-03-12,BOS,UTA,1.400000,3.050000,1
5,2024-03-12,MIL,SAC,1.704225,2.200000,0
6,2024-03-12,MIN,LAC,3.250000,1.359712,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-12,PHI,NYK,3.100000,1.387597,1.0
1,2024-03-12,IND,OKC,3.300000,1.350877,1.0
2,2024-03-12,WAS,MEM,1.769231,2.100000,1.0
3,2024-03-12,HOU,SAS,1.588235,2.420000,1.0
4,2024-03-12,BOS,UTA,1.400000,3.050000,1.0
5,2024-03-12,MIL,SAC,1.704225,2.200000,1.0
6,2024-03-12,MIN,LAC,3.250000,1.359712,0.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-12,PHI,NYK,3.100000,1.387597,1.0
1,2024-03-12,IND,OKC,3.300000,1.350877,1.0
2,2024-03-12,WAS,MEM,1.769231,2.100000,1.0
3,2024-03-12,HOU,SAS,1.588235,2.420000,1.0
4,2024-03-12,BOS,UTA,1.400000,3.050000,1.0
5,2024-03-12,MIL,SAC,1.704225,2.200000,1.0
6,2024-03-12,MIN,LAC,3.250000,1.359712,0.0
